In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import torch
import pdb

from pathlib import Path
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [3]:
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers import ProgressBar

In [4]:
from yelp.dataset import ProjectDataset
from yelp.trainer import YelpTrainer
from yelp.model import Classifier
from yelp.args import args

In [5]:
path = Path('./data/yelp')
review_csv = path/args.sample_file
scratch = path/args.workdir_name
vectorizer_path = scratch/args.vectorizer_fname
args.save_dir = scratch

df = pd.read_csv(review_csv)

In [6]:
args

Namespace(batch_size=1024, checkpointer_name='classifier', checkpointer_prefix='yelp', device='cuda:3', early_stopping_criteria=5, frequency_cutoff=25, learning_rate=0.001, num_epochs=100, sample_file='reviews_with_splits_lite.csv', save_dir=PosixPath('data/yelp/scratch'), save_every=2, save_total=5, vectorizer_fname='vectorizer.json', workdir_name='scratch')

In [7]:
# train_ds = ProjectDataset.load_data_and_create_vectorizer(df.loc[df['split'] == 'train'])
# train_ds.save_vectorizer(vectorizer_path)

In [8]:
train_df = df.loc[df['split'] == 'train']
train_ds = ProjectDataset.load_data_and_vectorizer(train_df, vectorizer_path)
vectorizer = train_ds.get_vectorizer()
train_dl = DataLoader(train_ds, args.batch_size, shuffle=True, drop_last=True)

val_df = df.loc[df['split'] == 'val']
val_ds = ProjectDataset.load_data_and_vectorizer(val_df, vectorizer_path)
val_dl = DataLoader(val_ds, args.batch_size, shuffle=True, drop_last=True)

In [9]:
classifier = Classifier(num_features=len((vectorizer).review_vocab))
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.5, patience=1)
loss_func = nn.BCEWithLogitsLoss()

In [10]:
def bce_logits_wrapper(output):
    y_pred, y = output
    y_pred = (torch.sigmoid(y_pred) > 0.5).long()
    return y_pred, y

In [11]:
pbar = ProgressBar(persist=True)
metrics = {'accuracy': Accuracy(bce_logits_wrapper), 'loss': Loss(loss_func)}

In [12]:
yelp_trainer = YelpTrainer(classifier, optimizer, loss_func, train_dl, val_dl, args, pbar, metrics)

In [13]:
yelp_trainer.run()

Epoch [1/100]: [38/38] 100%|██████████, loss=6.48e-01 [00:06<00:00]
Epoch [2/100]: [0/38]   0%|          , loss=5.57e-01 [00:00<?]

Epoch: 1
Training - Loss: 0.552, Accuracy: 0.839
Validation - Loss: 0.559, Accuracy: 0.829


Epoch [2/100]: [38/38] 100%|██████████, loss=5.31e-01 [00:06<00:00]
Epoch [3/100]: [0/38]   0%|          , loss=4.74e-01 [00:00<?]

Epoch: 2
Training - Loss: 0.477, Accuracy: 0.857
Validation - Loss: 0.487, Accuracy: 0.846


Epoch [3/100]: [38/38] 100%|██████████, loss=4.62e-01 [00:06<00:00]
Epoch [4/100]: [0/38]   0%|          , loss=4.34e-01 [00:00<?]

...

Epoch: 53
Training - Loss: 0.153, Accuracy: 0.951
Validation - Loss: 0.211, Accuracy: 0.921


Epoch [54/100]: [38/38] 100%|██████████, loss=1.55e-01 [00:06<00:00]
Epoch [55/100]: [0/38]   0%|          , loss=1.47e-01 [00:00<?]

Epoch: 54
Training - Loss: 0.152, Accuracy: 0.951
Validation - Loss: 0.211, Accuracy: 0.921


Epoch [55/100]: [38/38] 100%|██████████, loss=1.50e-01 [00:06<00:00]


Epoch: 55
Training - Loss: 0.150, Accuracy: 0.951
Validation - Loss: 0.210, Accuracy: 0.921
